In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GenAI para segmentação de clientes com BigQuery

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Segmentação de clientes é o processo de agrupar clientes base em características similares para ajudar no engajamento de uma marca ou de um produto com sua audiência. Isso pode ser feito usando uma variedade de fatores, como dados demográficos e comportamentais. Mecanismos de *machine learning* podem ser usados para automatizar o processo de segmentação de clientes, tornando-o mais eficiente e eficaz.

### BigQuery e o BigQueryML

O [BigQuery](https://cloud.google.com/bigquery/). é o serviço de base de dados gerenciado, serverless e escalável disponível na Google Cloud. O BigQuery permite gerenciar e analisar dados com recursos integrados, como *machine learning*, análise geoespacial e *business intelligence*. Para maiores detalhes, visite a [documentação oficial](https://cloud.google.com/bigquery/docs) do BigQuery.

O [BigQueryML](https://cloud.google.com/bigquery/docs/bqml-introduction?hl=pt-br) é o componente do BigQuery para criar e executar modelos de *machine learning* usando consultas com código GoogleSQL, permitindo profissionais de SQL criem modelos usando habilidades e ferramentas de SQL já existentes e sem a necessidade de movimentação de dados para fora do banco de dados. Para maiores detalhes, visite a [documentação oficial](https://cloud.google.com/bigquery/docs/bqml-introduction?hl=pt-br) do BigQueryML.


### Vertex AI PaLM API
A Vertex AI PaLM API, [lançada em 10 de maio de 2023](https://cloud.google.com/vertex-ai/docs/generative-ai/release-notes#may_10_2023), é desenvolvida com [PaLM 2]( https://ai.google/discover/palm2).

### Usando a API Vertex AI PaLM

Você pode interagir com a API Vertex AI PaLM usando os seguintes métodos:

* Use a UI da [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para testes rápidos e geração de comandos.
* Faça chamadas REST no Cloud Shell.
* Use o Python SDK em um notebook Jupyter

Este notebook se concentra no uso do Python SDK para chamar a Vertex AI PaLM API. Para obter mais informações sobre como usar o Vertex AI Studio sem escrever código, você pode explorar [Introdução às instruções da interface do usuário](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/getting-started/getting_started_ui.md)

Para obter mais informações, confira a [documentação sobre suporte de IA generativa para Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

Neste tutorial, você irá utilizar os serviços de Google Cloud para:

* Explorar dados do dataset público `bigquery-public-data.thelook_ecommerce.order_items`
* Treinar um modelo de segmentação utilizando o algoritmo `kmeans` no BigQueryML
* Realizar inferências no modelo treinado
* Utilizar as API do GenAI Studio para a geração de campanhas utilizando as informações do modelo do BigQueryML

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Studio
* BigQuery
* BigQueryML

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
[preços do BigQuery](https://cloud.google.com/bigquery/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

### Segurança de dados
**P: O Google usa dados de clientes para melhorar seus modelos de base?**
R: Não, o Google não usa dados de clientes para melhorar os modelos de fundação. Os dados do cliente são usados apenas para gerar uma resposta do modelo.

**P: Os funcionários do Google veem os dados que envio ao modelo?**
R: Não, os funcionários do Google não têm acesso aos dados do cliente e todos os dados são criptografados em trânsito, em uso e em repouso.

**P: O Google armazena algum dos dados do cliente que são enviados para o modelo?**
R: Não, o Google não armazena dados de clientes. No entanto, o Google pode armazenar em cache temporariamente os dados do cliente durante a solicitação, como pipeline de ajuste de prompt e uso em batch.

**P: O Google registra dados?**
R: Não, o Google não loga os dados dos clientes. Os logs do lado do sistema ajudam o Google a garantir a integridade e a disponibilidade do sistema.

### IA Responsável
LLMs podem traduzir linguagem, resumir texto, gerar escrita criativa, gerar código, chatbots e assistentes virtuais, além de complementar mecanismos de pesquisa e sistemas de recomendação. Ao mesmo tempo, como uma tecnologia em estágio inicial, seus recursos e usos em evolução podem criar aplicações incorretas, uso indevido e consequências não intencionais ou imprevistas. LLMs podem gerar resultados inesperados, incluindo texto ofensivo, insensível ou incorreto.

Além disso, a incrível versatilidade dos LLMs também é o que torna difícil prever exatamente que tipos de saídas não intencionais ou imprevistas eles podem produzir. Dados esses riscos e complexidades, a API PaLM foi projetada com os [AI Principles do Google](https://ai.google/principles/) em mente. No entanto, é importante que os desenvolvedores entendam e testem seus modelos para implantá-los com segurança e responsabilidade. Para ajudar os desenvolvedores, o Vertex AI Studio possui filtragem de conteúdo integrada e a API PaLM possui pontuação de atributo de segurança para ajudar os clientes a testar os filtros de segurança do Google e definir limites de confiança adequados para seu caso de uso e negócios. Consulte a seção [Filtros e atributos de segurança](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#safety_filters_and_attributes) para saber mais.

Quando a API PaLM é integrada ao caso de uso e contexto exclusivos de um cliente, considerações adicionais de IA Responsável e [limitações PaLM](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai #palm_limitations) precisam ser considerados. Incentivamos os clientes a usar *fairness*, interpretabilidade, privacidade e segurança [práticas recomendadas](https://ai.google/responsabilidades/responsible-ai-practices/).

## Primeiros Passos

### Instalando os SDK da Vertex AI

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

import sys
import pandas as pd
from typing import Union
from google.cloud import bigquery
from vertexai.language_models import TextGenerationModel

## Geração de texto com `text-bison@001`

O modelo de geração de texto da API PaLM que você usará neste notebook é `text-bison@001`. Já deixaremos seu objeto instanciado neste Jupyter notebook para uso futuro

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

#### Entendendo os dados disponíveis
Primeiramente, vamos entender os dados que utilizaremos. Vamos criar a *query* no dataset a ser utilizado:

In [ ]:
query = """
SELECT
  user_id,
  order_id,
  sale_price,
  created_at as order_created_date
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
AND CAST('2023-01-01 00:00:00' AS TIMESTAMP)
"""

Agora esta *query* será executada. Isso inclui a definição de um `client` do BigQuery, usando sua SDK e pegar o resultado da *query* e armazenar em um *dataframe* Pandas para uma melhor visualização aqui no ambiente de Jupyter Notebook:

In [ ]:
PROJECT_ID = "lucianomartins-demos-345000"  # @param {type:"string"}
client = bigquery.Client(project=PROJECT_ID)
df = client.query(query).to_dataframe()
df.head()

O próximo passo é criar um modelo do BigQueryML, utilizando o algoritmo `kmeans`.

Para isso, será utilizada a cláusula SQL `CREATE MODEL`, parte do BigQueryML, para iniciar um treinamento gerenciado de um modelo de *machine learning*. O *dataset de treinamento* será gerado para a partir de um select, em tempo de execução, e após o treinamento do modelo o BigQueryML irá armazenar seu modelo dentro do projeto do BigQuery em utilização.

Para maiores detalhes, consulte a documentação sobre [Criando modelos de machine learning no BigQuery ML](https://cloud.google.com/bigquery/docs/create-machine-learning-model?hl=pt-br).

Primeiro vamos criar um dataset no BigQuery para armazenar o modelo:

In [ ]:
project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id

dataset_id = f"{project_id}.segmentacao_de_clientes"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
try:
    dataset = client.create_dataset(dataset, timeout=30)
    print("Dataset criado com sucesso {}.{}".format(project_id, dataset.dataset_id))
except:
    print(f"O dataset {dataset_id} já existe")

Agora criaremos a *query* de criação do modelo:

In [ ]:
model_name = "modelo_kmeans"
query = f"""
CREATE OR REPLACE MODEL `{dataset_id}.{model_name}`
OPTIONS (
  MODEL_TYPE = "KMEANS",
  NUM_CLUSTERS = 5,
  KMEANS_INIT_METHOD = "KMEANS++",
  STANDARDIZE_FEATURES = TRUE )
AS (
SELECT * EXCEPT (user_id)
FROM (
  SELECT user_id,
    DATE_DIFF(CURRENT_DATE(), CAST(MAX(order_created_date) as DATE), day) AS days_since_order, -- RECENCY
    COUNT(order_id) AS count_orders, -- FREQUENCY
    AVG(sale_price) AS avg_spend -- MONETARY
  FROM (
    SELECT user_id,
      order_id,
      sale_price,
      created_at as order_created_date
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at BETWEEN CAST('2020-01-01 00:00:00' AS TIMESTAMP)
    AND CAST('2023-01-01 00:00:00' AS TIMESTAMP)
  )
  GROUP BY user_id, order_id
 )
)
"""

E realizaremos o treinamento do modelo. Este processo deve demorar mais ou menos 2 minutos:

In [ ]:
job_config = bigquery.QueryJobConfig()
client_result = client.query(query, job_config=job_config)
job_id = client_result.job_id
df = client_result.result().to_arrow().to_pandas()
print(f"Job de treinamento finalizado com sucesso: {job_id}")

#### Avaliando o modelo treinado

Como em todo processo de treinamento de modelos customizados, é importante realizarmos avaliações do modelo. Isso nos dará métricas numéricas e/ou qualitativas quanto ao resultado do treinamento.

Neste cenário, contaremos com a ajuda com BigQueryML para realizar a avaliação utilizando a cláusula SQL `ML.EVALUATE` e contaremos com as métricas [davies bouldin index](https://en.wikipedia.org/wiki/Davies%E2%80%93Bouldin_index) e [mean squared  error](https://en.wikipedia.org/wiki/Mean_squared_error) - De forma abstrata, quanto mais próximos de zero estes valores, melhor.

Primeiro criaremos a query de avaliação:

In [ ]:
query = f"""
SELECT *
FROM ML.EVALUATE(MODEL `{dataset_id}.{model_name}`)
"""

E então executaremos a query:

In [ ]:
df = client.query(query).to_dataframe()
df.head()

### Observando os centroids (ou agrupamentos criados pelo modelo `kmeans`)

Agora podemos observar um sumário de como o modelo agrupou os clientes em cinco diferente grupos. Primeiro vamos criar a query que fará essa análise:

In [ ]:
query = f"""
SELECT
  CONCAT('cluster ', CAST(centroid_id as STRING)) as centroid,
  avg_spend as average_spend,
  count_orders as count_of_orders,
  days_since_order
FROM (
  SELECT centroid_id, feature, ROUND(numerical_value, 2) as value
  FROM ML.CENTROIDS(MODEL `{dataset_id}.{model_name}`)
)
PIVOT (
  SUM(value)
  FOR feature IN ('avg_spend',  'count_orders', 'days_since_order')
)
ORDER BY centroid_id
"""

E agora podemos observar os grupos no resultado da execução da query:

In [ ]:
df = client.query(query).to_dataframe()
df.head()

### Enriquecendo a segmentação com Generative AI

Até neste ponto, foram realizados:

* a análise inicial dos dados
* o treinamento de um modelo de machine learning utilizando o BigQueryML
* a avaliação do modelo treinado
* a visualização dos grupos de clientes criados pelo modelo

Porém, ainda são atividades no mundo técnico dos dados. Como isso se torna mais próximo de um uso real de negócios?

Para isso, utilizaremos as informações dos grupos criados e as API do Vertex GenAI Studio para gerar conteúdos a partir do conhecimento que foi gerado aqui.

Primeiro vamos pegar as informações geradas na *query* dos grupos e salvar em uma variável chamada `clusters_info` de uma forma mais textual:

In [ ]:
df = client.query(query).to_dataframe()
df.to_string(header=False, index=False)

cluster_info = []
for i, row in df.iterrows():
  cluster_info.append("{0}, gasto médio ${2}, número de pedidos por pessoa {1}, dias desde a última compra {3}"
    .format(row["centroid"], row["count_of_orders"], row["average_spend"], row["days_since_order"]) )

print(str.join("\n", cluster_info))

Agora, com a ajuda da Vertex GenAI Studio, utilizando as informações descritivas dos grupos, iremos criar uma explicação mais textual de cada grupo:

In [ ]:
clusters = str.join("\n", cluster_info)

prompt = (f"""
Você é um estrategista de marca criativo. Dados os seguintes clusters, crie uma persona de marca criativa, um título cativante e a próxima ação de marketing, explicada passo a passo.

Clusters:
{clusters}

Para cada cluster:
* Título:
* Persona:
* Próximo passo de marketing:
""")

print(
        generation_model.predict(
            prompt,
            max_output_tokens=1024,
            temperature=0.55,
            top_p=0.8,
            top_k=40,
        ).text
)

E o mais interessante com esta abordagem é que a geração pode ser calibrada a partir do *prompt* que você encaminha ao modelo generativo. Um exemplo mais ainda criativo abaixo:

In [ ]:
cluster_info = str.join('\n', cluster_info)

prompt = (f"""
Você é um estrategista criativo, analise os seguintes clusters e crie \
persona de marca criativa para cada um que inclui os detalhes de qual personagem de Romeu e Julieta é \
provável que seja o favorito do grupo, um resumo de como isso se relaciona com seu comportamento de compra, \
e um título de e-mail espirituoso para campanha de marketing direcionada ao seu grupo.

Clusters:
{cluster_info}
""")

print(
    generation_model.predict(
        prompt,
        max_output_tokens=1024,
        temperature=0.45,
        top_p=0.8, top_k=40,
    ).text
)